# Week 6: Deep features for image retrieval

In this module, we focused on using deep learning to create non-linear features to improve the performance of machine learning. We also saw how transfer learning techniques can be applied to use deep features learned with one dataset to get great performance on a different dataset. We also built an iPython notebooks for both image retrieval and image classification tasks on real datasets.

In this assignment, we are going to build new image retrieval models and explore their results on different parts of our image dataset. These techniques will be used at the core of the intelligent application in your capstone project.

Follow the rest of the instructions on this page to complete your program. When you are done, instead of uploading your code, you will answer a series of quiz questions (see the quiz after this reading) to document your completion of this assignment. The instructions will indicate what data to collect for answering the quiz.

Learning outcomes

- Execute image retrieval code with the iPython notebook
- Use the .sketch_summary() method to view statistics of data
- Load and transform real, image data
- Build image retrieval models using nearest neighbor search and deep features
- Compare the results of various image retrieval models

In [1]:
import graphlab

[WARNING] Unable to write current GraphLab Create license to /Users/jcj/.graphlab/config. Ensure that this user account has write permission to /Users/jcj/.graphlab/config to save the license for offline use.
[INFO] This non-commercial license of GraphLab Create is assigned to chengjun@chem.ku.dk and will expire on January 27, 2017. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-29954 - Server binary: /usr/local/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1454664328.log
[INFO] GraphLab Server Version: 1.8.1
[WARNING] Unable to create session in specified location: '/Users/jcj/.graphlab/artifacts'. Using: '/var/tmp/graphlab-jcj/29954/tmp_session_52a46d2e-45d5-4813-8a4a-c0b56809e111'


In [2]:
graphlab.canvas.set_target('ipynb')

## Load and explore the data

Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [3]:
image_train = graphlab.SFrame('image_train_data/')

In [20]:
# homework
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


## Computing deep features for our images
The two lines below allow us to compute deep features. This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded.

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [4]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

## Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [5]:
knn_model = graphlab.nearest_neighbors.create(image_train,
                                             features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [8]:
cat = image_train[18:19]
cat['image'].show()

In [12]:
cat

id,image,label,deep_features,image_array
384,Height: 32 Width: 32,cat,"[1.04403531551, 0.0, 0.0,0.0, 0.0, 0.0, ...","[46.0, 45.0, 50.0, 47.0,45.0, 51.0, 45.0, 44.0, ..."


## Finding similar images to a car

In [13]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [15]:
cat_neighbors = cat_train.filter_by(knn_model.query(cat))

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 12.994ms     |
PROGRESS: | Done         |         | 100         | 214.84ms     |
PROGRESS: +--------------+---------+-------------+--------------+


In [16]:
cat_neighbors['image'].show()

In [17]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [18]:
show_neighbors(2000)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 21.198ms     |
PROGRESS: | Done         |         | 100         | 218.129ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [31]:
# homework
dog_train = image_train.filter_by(['dog'], 'label')
cat_train = image_train.filter_by(['cat'], 'label')
bird_train = image_train.filter_by(['bird'], 'label')
automobile_train = image_train.filter_by(['automobile'], 'label')

In [33]:
dog_model = graphlab.nearest_neighbors.create(dog_train,
                                               label='id',
                                               features=['deep_features'])

PROGRESS: Starting brute force nearest neighbors model training.


In [34]:
cat_model = graphlab.nearest_neighbors.create(cat_train,
                                               label='id',
                                               features=['deep_features'])

PROGRESS: Starting brute force nearest neighbors model training.


In [35]:
bird_model = graphlab.nearest_neighbors.create(bird_train,
                                               label='id',
                                               features=['deep_features'])

PROGRESS: Starting brute force nearest neighbors model training.


In [36]:
automobile_model = graphlab.nearest_neighbors.create(automobile_train,
                                               label='id',
                                               features=['deep_features'])

PROGRESS: Starting brute force nearest neighbors model training.


In [37]:
image_test = graphlab.SFrame('image_test_data/')

In [103]:
results1 = dog_model.query(image_test[0:1])
dog_train.filter_by(results1['reference_label'], 'id')['image'].show()#['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 30.755ms     |
PROGRESS: | Done         |         | 100         | 125.283ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [95]:
cat_query_results = cat_model.query(image_test[0:1], k=1)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 23.517ms     |
PROGRESS: | Done         |         | 100         | 111.114ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [96]:
cat_query_results

query_label,reference_label,distance,rank
0,16289,34.623719208,1


In [48]:
cat_query_results['distance'].mean()

36.15573070978294

In [49]:
dog_model.query(image_test[0:1])['distance'].mean()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 24.764ms     |
PROGRESS: | Done         |         | 100         | 101.403ms    |
PROGRESS: +--------------+---------+-------------+--------------+


37.77071136184156

In [51]:
image_test_cat = image_test.filter_by(['cat'], 'label')
image_test_dog = image_test.filter_by(['dog'], 'label')
image_test_bird = image_test.filter_by(['bird'], 'label')
image_test_automobile = image_test.filter_by(['automobile'], 'label')

In [53]:
cat_model.query(image_test_cat)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 370.014ms    |
PROGRESS: | Done         | 509000  | 100         | 396.438ms    |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,16289,34.623719208,1
0,45646,36.0068799284,2
0,32139,36.5200813436,3
0,25713,36.7548502521,4
0,331,36.8731228168,5
1,13094,33.8680579302,1
1,10883,34.4414068951,2
1,43295,34.8348775045,3
1,6304,34.9159478835,4
1,8302,35.1367177322,5


In [56]:
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)
dog_bird_neighbors = bird_model.query(image_test_dog, k=1)
dog_automobile_neighbors = automobile_model.query(image_test_dog, k=1)
dog_dog_neighbors = dog_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 388.727ms    |
PROGRESS: | Done         | 509000  | 100         | 409.949ms    |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 120000  | 25.1046     | 334.856ms    |
PROGRESS: 

In [57]:
dog_distances = graphlab.SFrame()

In [58]:
dog_distances['dog-automobile'] = dog_automobile_neighbors['distance']
dog_distances['dog-cat'] = dog_cat_neighbors['distance']
dog_distances['dog-bird'] = dog_bird_neighbors['distance']
dog_distances['dog-dog'] = dog_dog_neighbors['distance']

In [70]:
dog_distances

dog-automobile,dog-cat,dog-bird,dog-dog
41.9579761457,36.4196077068,41.7538647304,33.4773590373
46.0021331807,38.8353268874,41.3382958925,32.8458495684
42.9462290692,36.9763410854,38.6157590853,35.0397073189
41.6866060048,34.5750072914,37.0892269954,33.9010327697
39.2269664935,34.778824791,38.272288694,37.4849250909
40.5845117698,35.1171578292,39.1462089236,34.945165344
45.1067352961,40.6095830913,40.523040106,39.0957278345
41.3221140974,39.9036867306,38.1947918393,37.7696131032
41.8244654995,38.0674700168,40.1567131661,35.1089144603
45.4976929401,42.7258732951,45.5597962603,43.2422832585


In [71]:
dog_distances[0]

{'dog-automobile': 41.957976145712024,
 'dog-bird': 41.753864730351246,
 'dog-cat': 36.41960770675437,
 'dog-dog': 33.47735903726335}

In [75]:
def is_dog_correct(row):    
    if row['dog-dog'] < min(row['dog-cat'], row['dog-bird'], row['dog-automobile']):
        return 1
    else:
        return 0    

In [78]:
dog_distances.apply(is_dog_correct).sum()

678

In [85]:
678/1000.

0.678

In [84]:
len(image_test_automobile)

1000

In [81]:
len(dog_distances)

1000